# pre-procesamiento 01
El preprocesamiento consiste en acomodar los datos brw en una matriz que contenga 4096 filas
(una por cada canal) y n columnas (una por cada cuadro muestral).
Una vez organizados, se cortan fragmentos de 4096(n/m) donde m es divisor de n. El objetivo 
es obtener archivos de tamaño manejable tanto en tamaño muestral como en espacio de 
almacenamiento en disco y que no sature la memoria del equipo.
En este caso están almacenados en formato Float16

In [1]:
# path to Scripts
SCPpath = "/home/isabel/Dropbox/Proyects/Romero-Gutierrez/Scripts/Preprocesamiento/" 
cd( SCPpath )
push!( LOAD_PATH, "." ); # folder que continen los modulos. Correr desde ahí dentro.
#
using AllFunctions, BRW_things # modulo personal
using JLD # paquetes nativos
#

┌ Info: Precompiling BRW_things [top-level]
└ @ Base loading.jl:1273


In [2]:
#
Γ = OS( "linux" ); # establecer SO a usar...es para mi lap si?
Processpath = "/home/isabel/Desktop/200819/" # path donde están los experimentos
cd( Processpath ) 
BRWfiles = find_files(Processpath, "brw", Γ ); # todos los exp
BRWname = BRWfiles[ 1 ]; # solo uno, se puede hacer loop --------------------------------- !
println(string("Se procesará el archivo:", BRWname))
#

Se procesará el archivo:/home/isabel/Desktop/200819/acido_01.brw


In [3]:
# Crear ahi folder para guardar cachos (uno para cada exp)

Processpath = string( split( BRWname, "." )[ 1 ], Γ, "Process" ); 
checkpath( Processpath ); # checar si el folder para guardar los cachos existe
Varspath = string( split( BRWname, "." )[ 1 ], Γ, "Vars" );
checkpath( Varspath ); # checar si el folder para guardar las variables existe

#= variables del experimento para guardar al final con cada cacho, 
para evitar cargar el brw cada vez. En "dset" va el nombre del experimento =#

ν = brw_things( BRWname ); # variables útiles
ξ = ν[ "Chs" ]; # canales en numero, coord1, coord2
Σ = ν[ "dset" ]; # path al dataset (NO READ! to much)
ζ = ν[ "Factor" ]; # variables para...
ο = ν[ "Offset" ]; # conversión de voltaje
T = Int( size( Σ, 1 )/size( ξ, 1 ) ); # verdadero tiempo registrado
SR = round( ν[ "SamplingRate" ][ ], digits = 1 ); # .......Sampling Rate u k... 


In [4]:
# Se salva un diccionario con variables relevantes del archivo para uso posterior
delete!(ν, "dset")
Varsname = string( Varspath, Γ, "vars.jld" ); # Salvando las variables de interés
save( Varsname, "vars", ν );
println( " Vars file (vars.jld) saved. " );
#

 Vars file (vars.jld) saved. 


In [5]:
# tratando de encontrar un tamaño adecuado de cacho.

if isinteger( SR ) # así, se tienen cachos de 1 segundo
    if isinteger( T/SR )
        n_segments = Int.( T/SR );
        println( " Los cachos serán de 1 segundo " )
    end
elseif isinteger( T/floor( SR ) ) 
    n_segments = Int.( T/floor( SR ) );
    println( " Los cachos serán ~ 1 segundo " )
elseif isinteger( T/ceil( SR ) )
    n_segments = Int.( T/ceil( SR ) );
    println( " Los cachos serán ~ 1 segundo " )
else # si no, se hace un merequetengue
    div_T = div_n_ab( T );
    div_sec = div_T/SR;
    # rango de numero cachos que se quieren cortar, normalmente alto para tranajar a gusto
    hi = 4; lo = 2; # segundos
    if !isempty( div_T )
        # busca uno de los divisores de frames dentro del rango
        selected_divs = div_T[ findall( hi .>= div_sec .>= lo ) ];
    end
    if !isempty( selected_divs ) # si hubo, agarra el primero
        n_segments = Int(T/selected_divs[ 1 ]);
        println( " Los cachos serán de ", ( ( T/SR )/n_segments ) ," segundos " )
    else # si no hubo, uno predeterminado ya que
        n_segments = 60;
        println( " Los cachos serán de ", ( ( T/SR )/n_segments ) ," segundos " )
    end
end
#

 Los cachos serán de 2.0 segundos 


In [6]:
# numero de espacios que ocupa el numero de segundos de duracion del experimento
n_char = length( string( Int( floor( T/SR ) ) ) ); 
# numero de espacios que ocupa el numero de cachos 
n_bins = length( string( n_segments ) );
ω = Int(ceil( T/n_segments ) ); # numero de frames finales (tamaño del cacho en frames)
ε = zeros( Int, n_segments, 2 ); # preallocate
ε[ :, 1 ] = collect( 1:ω:T ); # inicio y 
ε[ :, 2 ] = ε[ :, 1 ] .+ ω .- 1; # fin en frames de cada cacho (para cortar)
if !isinteger( T/n_segments )
    println(" El ultimo cacho es más chico ")
    ε[ end, 2 ] = T; # fin en frames de cada cacho (para cortar)
end
#

In [7]:
#
for i = 1:n_segments # numero de B a cortar ( 1->4096, 1->ω )
    BIN = zeros( size( ξ, 1 ), ω ); # preallocate
    #= valores correspondientes al BIN especifico. 
    El canal 1,1 tiene el frame 1, 4097, 8193...etc =#
    β = collect( ( ε[ i, 1 ] - 1 ):1:ε[ i, 2 ] ); 
    for j = 1:ω
        # saca esos frames del machote seguido Σ, y ponlos en array en BIN
        BIN[ :, j ] = Σ[ ( β[ j ]*size( ξ, 1 ) ) + 1:( size( ξ, 1 )*β[ j + 1 ] ) ];
    end
    BIN = Float16.( ( BIN.*ζ ) .+ ο ); # conversion a volaje
    ini = lpad( string( Int(floor( ( β[ 1 ] )/SR ) ), "s" ), n_char + 1, "0" );
    eni = lpad( string( Int( floor( ( β[ end ] + 1 )/SR ) ), "s" ), n_char + 1, "0" );
    bin_time = string( ini,"-",eni );
    BINname = string( Processpath, Γ, "BIN", lpad(i, n_bins, "0" ), "_", bin_time, ".jld" );   
    save( BINname, "data", BIN );
    println( lpad(i, n_bins, "0" ), " of ", n_segments, " saved" );
end
#

01 of 90 saved
